# Uso de Script para Escuchar Radio con RTL-SDR
Este script permite sintonizar y escuchar radio utilizando un dispositivo RTL-SDR. Se configura la frecuencia de escucha y la posibilidad de guardar el audio. La recepción se realiza mediante muestras de radiofrecuencia, procesadas para generar una señal de audio que se reproduce en tiempo real.

En primer lugar importamos las librerías necesarias para el desarrollo del proyecto.

In [ ]:
# Importar librerías necesarias
import sys
from typing import List
from rtlsdr import RtlSdr
import argparse
import datetime
import numpy as np
import pyaudio
import scipy.signal as sg
import signal
import wave

Inicializamos las variables que vamos a utilizar y la salida de audio para nuestro dispositivo. Aquí indicamos la frecuencia que queremos sintonizar y si queremos guardar el audio en un archivo.

In [ ]:
SampleStream = List[float]
AudioStream = List[int]
audio_rate = 48000
save = False
freq = 98000000
# Inicializar dispositivo de salida de audio
audio_output = pyaudio.PyAudio().open(format=pyaudio.paInt16, channels=1, rate=audio_rate, output=True)
audio_data = []

: 

Esta función se encargará de guardar los datos de audio en un archivo. Para ello, se le pasa como parámetro el nombre del archivo y los datos de audio hasta el momento en el que se detiene la ejecución del script.

In [ ]:
def save_audio_data_to_file():
    filename = f"audio_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.mp3"
    audio_data_bytes = np.array(audio_data).tobytes()
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(1)  # Número de canales (1 para mono, 2 para estéreo)
        wf.setsampwidth(2)  # Ancho en bytes de cada muestra
        wf.setframerate(48000)  # Tasa de muestreo en Hz
        wf.writeframes(audio_data_bytes)

**Función para manejar la interrupción del teclado (Ctrl+C)**: Gracias a esta función, podremos interrumpir en todo momento la ejecución del script, deteniendo la reproducción de audio y cerrando el dispositivo RTL-SDR. En caso de que hayamos indicado en la configuración que queremos guardar el audio, se guardará el archivo y se cerrará. 

In [ ]:
def signal_handler(sig, frame):
    global sdr
    sdr.cancel_read_async()
    if save:
        save_audio_data_to_file()
    print("Ctrl+C detectado. Liberando recursos...")
    sdr.close()

Con esta función controlamos la reproducción de audio, redireccionándolo a la salida de audio de nuestro dispositivo. En caso de que queramos que se guarde el audio al finalizar la ejecución, iremos añadiendo los datos de audio a una lista.

In [ ]:
def stream_audio(data: AudioStream):
    if save:
        audio_data.append(data)
    audio_output.write(data)

La función *process* se encarga de transformar las muestras de radiofrecuencia en una señal de audio que puede ser reproducida.

In [ ]:
def process(samples: SampleStream, sdr: RtlSdr) -> None:
    sample_rate_fm = 240000
    iq_comercial = sg.decimate(samples, int(sdr.get_sample_rate()) // sample_rate_fm)

    angle_comercial = np.unwrap(np.angle(iq_comercial))
    demodulated_comercial = np.diff(angle_comercial)
    audio_signal = sg.decimate(demodulated_comercial, sample_rate_fm // audio_rate, zero_phase=True)
    audio_signal = np.int16(14000 * audio_signal)

    stream_audio(audio_signal.astype("int16").tobytes())

Finalmente, inicializamos el dispositivo RTL-SDR y comenzamos a escuchar la frecuencia indicada en la configuración. 

In [ ]:
sdr =  RtlSdr()
signal.signal(signal.SIGINT, signal_handler) 
try:
    sdr.rs = 240000
    sdr.fc = freq
    sdr.gain = 'auto'
    sdr.err_ppm = 40
    sdr.read_samples_async(process, int(sdr.get_sample_rate()) // 16)
finally:
    sys.exit(0)